<h1>Problem 1: Generating Random Boolean Functions</h1>

In [ ]:
import random

def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function f: {0,1}^4 -> {0,1}
    that is either constant or balanced.
    """
    # randomly choose: constant (true) or balanced (false)
    if random.choice([True, False]):
        # constant
        
        # randomly pick the constant value , so always return False or always return True
        value = random.choice([False, True])
        
        #ignores inputs and always returns the same value
        def f(a, b, c, d):
            return value  # use the value we picked above 
        
        return f
    
    else:
        # balanced
        
        # Generate 16 input combinations for 4 variables
        # This creates: (F,F,F,F), (F,F,F,T), (F,F,T,F), ..., (T,T,T,T)
        inputs = [(a, b, c, d)
                  for a in (False, True)
                  for b in (False, True)
                  for c in (False, True)
                  for d in (False, True)]
        
        # select exactly 8 of the 16 inputs to return True rest return false
        true_inputs = set(random.sample(inputs, 8))
        
        # returns True only for the selected 8 inputs
        def f(a, b, c, d):
            # Check if current input is one of the "True" inputs
            return (a, b, c, d) in true_inputs  
        
        return f

<h1>Problem 2: Classical Testing for Function Type</h1>

<h1>Problem 3: Quantum Oracles</h1>

<h1>Problem 4: Deutsch's Algorithm with Qiskit</h1>

<h1>Problem 5: Scaling to the Deutsch–Jozsa Algorithm</h1>